In [62]:
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import datetime as dt
from pathlib import Path
from newsapi import NewsApiClient
from nltk.corpus import stopwords, reuters
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import os
import csv
import hvplot.pandas
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samuelarciniega/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [63]:
load_dotenv()

True

In [64]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [65]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["AAPL", "FB", "GOOGL", "SPY"]
tech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
tech_ticker.dropna(inplace=True)
tech_ticker.head(20)

AAPL                                          \
                             open      high      low     close     volume   
time                                                                        
2021-01-04 00:00:00-05:00  133.52  133.6116  126.760  129.4100  123353442   
2021-01-05 00:00:00-05:00  128.89  131.7400  128.430  130.9500   85139362   
2021-01-06 00:00:00-05:00  127.72  131.0499  126.382  126.5800  142064548   
2021-01-07 00:00:00-05:00  128.36  131.6300  127.860  130.8500   95314687   
2021-01-08 00:00:00-05:00  132.43  132.6300  130.230  131.9900   93211863   
2021-01-11 00:00:00-05:00  129.19  130.1700  128.500  128.9800   84845947   
2021-01-12 00:00:00-05:00  128.50  129.6900  126.860  128.8199   83381829   
2021-01-13 00:00:00-05:00  128.76  131.4500  128.490  130.8900   77928722   
2021-01-14 00:00:00-05:00  130.80  131.0000  128.760  128.9200   78672922   
2021-01-15 00:00:00-05:00  128.78  130.2242  127.000  127.0700   97478662   
2021-01-19 00:00:00-05:00  127.78  128.7100  126.938  127.8200   77372754   
2021-01-20 00:00:00-05:00  128.66  132.4900  128.550  131.9700   90745056   
2021-01-21 00:00:00-05:00  133.80  139.6700  133.590  136.8700  107349916   
2021-01-22 00:00:00-05:00  136.28  139.8500  135.020  138.9699  104455613   
2021-01-25 00:00:00-05:00  143.07  145.0900  136.540  142.9000  142335837   
2021-01-26 00:00:00-05:00  143.60  144.3000  141.370  143.1850   85816874   
2021-01-27 00:00:00-05:00  143.43  144.3000  140.410  142.5300  110342303   
2021-01-28 00:00:00-05:00  139.52  141.9900  136.700  137.0400  127192871   
2021-01-29 00:00:00-05:00  135.83  136.7400  130.210  131.8300  162479747   
2021-02-01 00:00:00-05:00  133.75  135.3800  130.930  134.1000   96660055   

                                 FB                                      \
                               open    high       low   close    volume   
time                                                                      
2021-01-04 00:00:00-05:00  274.7800  275.00  265.2000  268.94  12460207   
2021-01-05 00:00:00-05:00  268.2900  272.40  268.2111  270.97   8053296   
2021-01-06 00:00:00-05:00  262.0000  267.75  260.0100  263.24  21446415   
2021-01-07 00:00:00-05:00  265.9000  271.61  264.7800  268.63  13968342   
2021-01-08 00:00:00-05:00  268.3100  268.95  263.1800  267.54  16730340   
2021-01-11 00:00:00-05:00  260.4800  263.47  255.9000  256.81  27236030   
2021-01-12 00:00:00-05:00  256.6300  259.72  250.3000  250.91  24074761   
2021-01-13 00:00:00-05:00  251.5500  253.95  249.2000  251.64  16896303   
2021-01-14 00:00:00-05:00  253.4000  255.03  244.6100  245.71  27927382   
2021-01-15 00:00:00-05:00  247.9000  253.86  247.1600  251.36  22675147   
2021-01-19 00:00:00-05:00  256.9000  262.20  252.7201  261.08  25191870   
2021-01-20 00:00:00-05:00  268.9300  270.32  263.6000  267.35  22539886   
2021-01-21 00:00:00-05:00  269.2600  273.60  267.4901  272.89  17034362   
2021-01-22 00:00:00-05:00  272.0100  278.47  272.0000  274.53  19132340   
2021-01-25 00:00:00-05:00  278.1400  280.10  271.5100  278.00  17646954   
2021-01-26 00:00:00-05:00  278.1398  285.39  277.8100  282.02  17018388   
2021-01-27 00:00:00-05:00  282.5300  283.45  268.1201  272.39  27484121   
2021-01-28 00:00:00-05:00  277.1800  286.79  264.7000  265.00  35383730   
2021-01-29 00:00:00-05:00  265.3000  266.56  254.8500  258.33  28139673   
2021-02-01 00:00:00-05:00  259.5200  264.17  254.9100  261.98  19629047   

                              GOOGL                                          \
                               open       high        low    close   volume   
time                                                                          
2021-01-04 00:00:00-05:00  1760.000  1762.4899  1707.1400  1726.39  1476533   
2021-01-05 00:00:00-05:00  1725.090  1746.8300  1716.9000  1739.26   831935   
2021-01-06 00:00:00-05:00  1700.255  1743.9650  1696.1014  1723.38  2091919   
2021-01-07 00:00:00-05:00  1726.760  1777.8200  1726.7600

In [66]:
timeframe = "1D"
start_date = pd.Timestamp("2021-06-02", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-06-29", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["AAPL"]
appl_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
appl_ticker.dropna(inplace=True)
appl_ticker.head(20)

AAPL                                       
                             open     high       low     close    volume
time                                                                    
2021-06-02 00:00:00-04:00  124.29  125.240  124.0500  125.0900  51393692
2021-06-03 00:00:00-04:00  124.66  124.850  123.1300  123.5550  68331470
2021-06-04 00:00:00-04:00  124.07  126.160  123.8500  125.8800  63613126
2021-06-07 00:00:00-04:00  126.18  126.320  124.8321  125.9100  62351961
2021-06-08 00:00:00-04:00  126.62  128.460  126.2101  126.7400  67231488
2021-06-09 00:00:00-04:00  127.21  127.750  126.5200  127.1600  46854275
2021-06-10 00:00:00-04:00  127.02  128.190  125.9400  126.1100  63377192
2021-06-11 00:00:00-04:00  126.53  127.440  126.1000  127.3521  46206013
2021-06-14 00:00:00-04:00  127.82  130.490  127.0700  130.4900  73169843
2021-06-15 00:00:00-04:00  129.88  130.600  129.3900  129.7400  53587032
2021-06-16 00:00:00-04:00  130.29  130.890  128.4610  130.1400  81577235
2021-06-17 00:00:00-04:00  129.80  132.550  129.6500  131.7800  82434812
2021-06-18 00:00:00-04:00  130.65  131.510  130.2400  130.4000  78114203
2021-06-21 00:00:00-04:00  130.32  132.410  129.2100  132.2900  63245869
2021-06-22 00:00:00-04:00  132.10  134.080  131.6200  133.9700  61664813
2021-06-23 00:00:00-04:00  133.78  134.320  133.2300  133.7000  54089975
2021-06-24 00:00:00-04:00  134.45  134.640  132.9300  133.4000  57263070
2021-06-25 00:00:00-04:00  133.47  133.890  132.8100  133.1100  42633100
2021-06-28 00:00:00-04:00  133.42  135.245  133.3500  134.7500  53815255
2021-06-29 00:00:00-04:00  134.82  136.490  134.3500  136.3300  55081959

In [68]:
apple_df = pd.DataFrame()
apple_df["Apple"] = appl_ticker["AAPL"]["close"]
#apple_df[" "] = apple_df["Date"]
#apple_df.index = apple_df.index.date
apple_df.head(20)

,Apple
time,
2021-06-02 00:00:00-04:00,125.0900
2021-06-03 00:00:00-04:00,123.5550
2021-06-04 00:00:00-04:00,125.8800
2021-06-07 00:00:00-04:00,125.9100
2021-06-08 00:00:00-04:00,126.7400
2021-06-09 00:00:00-04:00,127.1600
2021-06-10 00:00:00-04:00,126.1100
2021-06-11 00:00:00-04:00,127.3521
2021-06-14 00:00:00-04:00,130.4900


In [46]:
api_key = os.getenv("news_api")
print(api_key)

6dde5b0486654982bdc6ebe38ab427a4


In [47]:
newsapi = NewsApiClient(api_key=api_key)

In [48]:
tech_news = newsapi.get_everything(q = "Technology & Regulation", language="en", page_size=100, sort_by="relevancy")
print(f"Total articles about Inflation: {tech_news['totalResults']}")
tech_news["articles"][0]

Total articles about Inflation: 3657


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Igor Bonifacic',
 'title': 'Automakers must report crashes involving self-driving and driver-assist systems',
 'description': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within one day of learning of an accident. In an\xa0order\xa0spotted by\xa0…',
 'url': 'https://www.engadget.com/nhtsa-automated-driving-system-crash-report-174552853.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-images/2020-01/1e88d580-3edd-11ea-8a57-ce70dbfb73e6',
 'publishedAt': '2021-06-29T17:45:52Z',
 'content': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within… [+1815 chars]'}

In [49]:
sentiments = []
for article in tech_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]

        sentiments.append({
            "Text": text,
            "Date": date,
            "Compound":compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu
        })
    except AttributeError:
        pass
tech_df = pd.DataFrame(sentiments)

columns = ["Date", "Text", "Compound", "Positive", "Negative", "Neutral"]
tech_df = tech_df[columns]
#apple_df.index = apple_df.index.date
tech_df.index = tech_df.Date
tech_df.head(100)

,Date,Text,Compound,Positive,Negative,Neutral
Date,,,,,,
2021-06-29,2021-06-29,The National Highway Traffic Safety Administra...,0.4215,0.091,0.000,0.909
2021-06-23,2021-06-23,"TOKYO, June 23 (Reuters) - The Japanese govern...",0.2023,0.055,0.000,0.945
2021-06-28,2021-06-28,Telefonica's Chairman and CEO Jose Maria Alvar...,0.2023,0.060,0.000,0.940
2021-06-09,2021-06-09,Representations of cryptocurrencies including ...,0.0000,0.000,0.000,1.000
2021-06-18,2021-06-18,The UK’s chief data protection regulator has w...,-0.5859,0.000,0.130,0.870
...,...,...,...,...,...,...
2021-06-14,2021-06-14,Nvidia announced last fall that it was officia...,0.0000,0.000,0.000,1.000
2021-06-04,2021-06-04,"This week, King county, Washington, became the...",-0.5574,0.000,0.098,0.902
2021-06-29,2021-06-29,Edge computing can often deliver the same resu...,-0.6012,0.000,0.169,0.831


In [51]:
apple_joined = pd.concat([apple_df, tech_df], axis="columns", join="right",)
apple_joined

ValueError: Only can inner (intersect) or outer (union) join the other axis

In [59]:
appl_df = pd.DataFrame(appl_ticker)

In [69]:
signals_df = apple_df.loc[:, ['time', 'Apple']].copy()
signals_df = signals_df.set_index("time", drop=True)
short_window = 50
long_window = 100
#signals_df = signals_df.set_index("Date", drop=True)
signals_df["SMA50"] = signals_df["Apple"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Apple"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] < signals_df["SMA100"][short_window:], 1.0, 0.0
)
signals_df['Entry/Exit'] = signals_df['Signal'].diff()
signals_df.tail(10)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['time'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"